# Example: Multiport atom interferometer

In [1]:
import numpy as np

import aisim as ais

from functools import partial

We simulate a Mach-Zehnder atom interferometer sequence, including all six output ports that are created due to imperfect beam splitter and mirror pulses.

Each of these output ports is represented by an entry in each of the atoms' state vector properties (e.g. `state_ket`). The index of each port is illustrated in the following schematic:

![title](multiport_ai.jpeg)

That means that the excited and ground states after the first beamsplitter have indices 0 and 5, respectively.

First, we establish all relevant objects and parameters for the atomic sample and the experimental apparatus:

In [2]:
t_det = 778e-3  # time of the detection in s
r_det = 5e-3  # size of detected region in x-y plane
det = ais.SphericalDetector(t_det, r_det=r_det)  # set detection region

center_rabi_freq = 2 * np.pi * 12.5e3
r_profile = 29.5e-3 / 2  # 1/e^2 beam radius in m
intensity_profile = ais.IntensityProfile(r_profile, center_rabi_freq)
wave_vectors = ais.Wavevectors(k1=8.052945514e6, k2=-8.052802263e6)

# Creating an atomic ensemble, initially in the ground state |g, v_0, x_0 + v_0*t>
# and continuing only with the atoms that are eventually detected
atoms = ais.create_random_ensemble(
    int(1e4),
    x_dist=partial(ais.dist.position_dist_gaussian, std=3.0e-3),
    y_dist=partial(ais.dist.position_dist_gaussian, std=3.0e-3),
    z_dist=partial(ais.dist.position_dist_gaussian, std=3.0e-3),
    vx_dist=partial(ais.dist.velocity_dist_from_temp, temperature=2.5e-6),
    vy_dist=partial(ais.dist.velocity_dist_from_temp, temperature=2.5e-6),
    vz_dist=partial(
        ais.dist.velocity_dist_from_temp, temperature=100e-9
    ),  # only atoms within a narrow velocity class are pre-selected
    state_kets=[0, 0, 0, 0, 0, 1],
)
atoms = det.detected_atoms(atoms)

Next, we create the three propagators that comprise the atom interferometer and a propagator modeling free evolution of the atoms:

In [3]:
# length of a beamspliter pulse
tau = 23e-6

beam_splitter1 = ais.SpatialSuperpositionTransitionPropagator(
    tau,
    n_pulse=1,
    n_pulses=3,
    intensity_profile=intensity_profile,
    wave_vectors=wave_vectors,
)

mirror = ais.SpatialSuperpositionTransitionPropagator(
    2 * tau,
    n_pulse=2,
    n_pulses=3,
    intensity_profile=intensity_profile,
    wave_vectors=wave_vectors,
)

beam_splitter2 = ais.SpatialSuperpositionTransitionPropagator(
    tau,
    n_pulse=3,
    n_pulses=3,
    intensity_profile=intensity_profile,
    wave_vectors=wave_vectors,
)

free_prop = ais.FreePropagator(260e-3)

To illustrate the simulation, we propagate the atoms step by step. Initially, all atoms are in the ground state:

In [4]:
[np.mean(atoms.state_occupation(i)) for i in range(6)]

[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]

First, the atoms are subjected to the first beamsplitter pulse that transfers atoms into a roughly equal superposition between ground and excited state, i.e. indices 0 and 5:

In [5]:
atoms = beam_splitter1.propagate(atoms)

In [6]:
[np.mean(atoms.state_occupation(i)) for i in range(6)]

[0.4617680981617182, 0.0, 0.0, 0.0, 0.0, 0.5382319018382818]

After free propagation a mirror pulse ideally transfers all atoms from the excited state (index 0) to the ground state (index 3) and from the ground state (index 5) to the excited state (index 2):

In [7]:
atoms = free_prop.propagate(atoms)
atoms = mirror.propagate(atoms)

In [8]:
[np.mean(atoms.state_occupation(i)) for i in range(6)]

[0.1188964472677993,
 0.0,
 0.3746771445733455,
 0.34287165089391897,
 0.0,
 0.16355475726493632]

Finally, the second beam splitter pulse couples three pairs of states as indicated in the schematic above (indices 0 ↔ 1, 2 ↔ 3, 4 ↔ 5).

In [9]:
atoms = free_prop.propagate(atoms)
atoms = beam_splitter2.propagate(atoms)

 Indices 2 and 3 form the output of the closed interferometer that are of relevance for us:

In [10]:
[np.mean(atoms.state_occupation(i)) for i in [2, 3]]

[0.011844398649651894, 0.7057043968176124]

From this, we infer a fringe contrast of roughly 0.71 - 0.01 = 70% which aligns well with experimental results.